<h1>SI 670 Kaggle Competition 4</h1>

Team GSIs

In [182]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [115]:
tf.config.list_physical_devices('GPU')

[]

<h2>Load datasets</h2>

In [2]:
len(list(os.listdir("../data/images_train/bathroom"))), len(list(os.listdir("../data/images_train/kitchen")))

(301, 315)

In [3]:
data = tf.keras.utils.image_dataset_from_directory("../data/images_train", image_size = (256, 256))

Found 616 files belonging to 2 classes.


In [4]:
X, y = zip(*data)

X = np.concatenate(X)
y = np.concatenate(y)

In [5]:
len(X), len(y)

(616, 616)

<h2>Preprocess Images</h2>

In [6]:
X = np.apply_along_axis(lambda s: s / 255.0, axis = 1, arr = X)

<h2>Split</h2>

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state = 42, test_size = 0.2)

<h2>Train Model</h2>

In [112]:
model = Sequential([
    tf.keras.layers.Conv2D(256, (3, 3), activation = 'relu', input_shape = (256, 256, 3)), 
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'), 
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation = 'relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(64), 
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [113]:
loss = BinaryCrossentropy()
model.compile(optimizer = 'adam', loss = loss, metrics = ['accuracy'])

In [116]:
early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0.005, patience = 3)

model_history = model.fit(X_train, y_train, epochs = 50, batch_size = 8, 
                          callbacks = [early_stopping], 
                          validation_data = (X_val, y_val))

Epoch 1/50
50/50 [==============================] - 287s 6s/step - loss: 0.6904 - accuracy: 0.5318 - val_loss: 0.6400 - val_accuracy: 0.5657
Epoch 2/50
50/50 [==============================] - 297s 6s/step - loss: 0.6145 - accuracy: 0.6692 - val_loss: 0.6910 - val_accuracy: 0.5657
Epoch 3/50
50/50 [==============================] - 294s 6s/step - loss: 0.4966 - accuracy: 0.7684 - val_loss: 0.5135 - val_accuracy: 0.7980
Epoch 4/50
50/50 [==============================] - 275s 6s/step - loss: 0.4796 - accuracy: 0.7812 - val_loss: 0.3325 - val_accuracy: 0.8485
Epoch 5/50
50/50 [==============================] - 303s 6s/step - loss: 0.3020 - accuracy: 0.8855 - val_loss: 0.4808 - val_accuracy: 0.7475
Epoch 6/50
50/50 [==============================] - 280s 6s/step - loss: 0.2908 - accuracy: 0.8804 - val_loss: 0.4047 - val_accuracy: 0.8485
Epoch 7/50
50/50 [==============================] - 294s 6s/step - loss: 0.2133 - accuracy: 0.9338 - val_loss: 0.3851 - val_accuracy: 0.8485


<h2>Test Model</h2>

In [117]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 26s 6s/step - loss: 0.4103 - accuracy: 0.8548


[0.41032207012176514, 0.8548387289047241]

In [118]:
model.evaluate(X_val, y_val)

4/4 [==============================] - 20s 5s/step - loss: 0.3851 - accuracy: 0.8485


[0.38506606221199036, 0.8484848737716675]

<h2>Make Prediction</h2>

In [172]:
pred_data = tf.keras.utils.image_dataset_from_directory("../data/images_test", image_size = (256, 256), labels = None)
pred_data = pred_data.unbatch()
pred_images = [img / 255.0 for img in pred_data]

Found 149 files belonging to 1 classes.


In [176]:
predictions = model.predict(np.array(pred_images))

5/5 [==============================] - 34s 6s/step


<h2>Format predictions for submission</h2>

In [180]:
img_files = os.listdir("../data/images_test")
img_ids = [f.split(".")[0] for f in img_files]

In [190]:
class_predictions = np.where(predictions > 0.5, 1, 0)

In [192]:
class_predictions[0].shape

(1,)

In [183]:
df = pd.DataFrame({'id': img_ids, 'class': predictions})

ValueError: Per-column arrays must each be 1-dimensional